In this part, we choose the first three company in boston and build the databases.There are 10 tables in our databases, which are employee, company, twitterUser, person, tweet, tweetRhashtag, hashtag, people, peopleRjob, and Job. tweetRhashtag and peopleRjob are two relationship table, which transit information. 

# API part

In [616]:
#import pandas library
import pandas as pd

In [617]:
#Read the csv from last assignment
raw_csv = pd.read_csv('Boston-Data Visualization-job-results.csv')

In [618]:
raw_csv.head()

,Name,Company,State,City,Salary,Location,Url
0,Data Visualization Engineer,EtQ,MA,Burlington,$89k-$154k,"Burlington, MA",https://www.glassdoor.com/partner/jobListing.h...
1,"Director, Data Scientist – Visualization, Nova...",Novartis,MA,Cambridge,$128k-$208k,"Cambridge, MA",https://www.glassdoor.com/partner/jobListing.h...
2,AI & Data - Data Analytics and Insights Lead,Bose,MA,Boston,$50k-$95k,"Boston, MA",https://www.glassdoor.com/partner/jobListing.h...
3,Data Engineer Intern,Health Catalyst,MA,Cambridge,NaN,"Cambridge, MA",https://www.glassdoor.com/partner/jobListing.h...
4,Data Scientist,IT Concepts,MA,Boston,$101k-$135k,"Boston, MA",https://www.glassdoor.com/partner/jobListing.h...


In [619]:
#Choose the first three companies
bos_csv = raw_csv.iloc[0:3]

In [620]:
bos_csv

,Name,Company,State,City,Salary,Location,Url
0,Data Visualization Engineer,EtQ,MA,Burlington,$89k-$154k,"Burlington, MA",https://www.glassdoor.com/partner/jobListing.h...
1,"Director, Data Scientist – Visualization, Nova...",Novartis,MA,Cambridge,$128k-$208k,"Cambridge, MA",https://www.glassdoor.com/partner/jobListing.h...
2,AI & Data - Data Analytics and Insights Lead,Bose,MA,Boston,$50k-$95k,"Boston, MA",https://www.glassdoor.com/partner/jobListing.h...


In [621]:
#Only keep the job name, company name and the salary
keep_col = ['Name','Company','Salary']
bos_csv = bos_csv[keep_col]
bos_csv

,Name,Company,Salary
0,Data Visualization Engineer,EtQ,$89k-$154k
1,"Director, Data Scientist – Visualization, Nova...",Novartis,$128k-$208k
2,AI & Data - Data Analytics and Insights Lead,Bose,$50k-$95k


In [622]:
bos_csv.to_csv("bos.csv")

In [623]:
#import librarys and use the api to scrap information 
import tweepy
import json
import datetime
import warnings
warnings.filterwarnings('ignore')
consumerKey = '9oKAuFyGssLe8L9aCSTI0MRNU'
consumerSecret = 'UQJZvzAHF0no4K92nu4uoQWvt9c8PfCedNnBxMzbAlZnOe7FkF'
ACCESS_TOKEN = '520412304-CAlp1M4JafYn4LohYxqWOexB9TpmEKYqdkbz63D5'
ACCESS_SECRET = 'ijiCxaPWt0O314ljXWMywj1oiu6hvB2i42VvjQFaSfMPZ'
auth = tweepy.OAuthHandler(consumer_key=consumerKey, 
    consumer_secret=consumerSecret)

#Connect to the Twitter API using the authentication
api = tweepy.API(auth)

In [624]:
# tweet to hashtag list, we will use it later
def tweettohashtaglist(tweet):
    hashtags = tweet._json.get("entities").get("hashtags")
    taglist = []
    for hashtag in hashtags:
        taglist.append(hashtag.get("text"))
    return taglist

In [625]:
# anyname to 30 days tweets list, we will use it later
def anynameto30daystweets(api, anyname):
    pgcount = 1
    allflag = False
    timenow = datetime.datetime.now()
    tweetlist = []
    while not allflag:
        tweets = api.user_timeline(anyname, page = pgcount, tweet_mode = 'extended')
        for tweet in tweets:
            if (timenow - tweet.created_at).days < 60:
                tweetlist.append(tweet)
            else:
                 allflag = True
        if not allflag:
            pgcount+=1
        else:
            return tweetlist

In [626]:
# hashtag to tweetlist, we will use it later
def hashtagtotweetlist(hashtag):
    tweets = []
    for tweet in tweepy.Cursor(api.search, q='#' + hashtag).items(2000):
        tweets.append(tweet)
    return tweets

# Setup the pandas table 

Here we initialize the tables that we will use later. 

In [627]:
import pandas as pd
company = pd.DataFrame(columns=["companyId", "companyName", "twitterUserId"])
people = pd.DataFrame(columns=["personId"])
peopleRjob = pd.DataFrame(columns=["prjId", "personId", "jobId"])
job = pd.DataFrame(columns=["jobId", "jobName"])
hashtag = pd.DataFrame(columns=["hashtagId", "text"])
person = pd.DataFrame(columns=["personId", "personName", "twitterUserId"])
employee = pd.DataFrame(columns=["empId", "personId", "jobId", "companyId"])
tweet = pd.DataFrame(columns=["tweetId", "twitterUserId", "content", "favoriteCount", "createdTime"])
tweetRhashtag = pd.DataFrame(columns=["trhId", "tweetId", "hashtagId"])
twitteruser = pd.DataFrame(columns=["twitterUserId", "name", "screenname", "location", "followersCount", "tweetsCount"])

In [628]:
# new twitter user appending function
# anyname to user
def anynametouser(anyname):
    user = api.get_user(anyname)
    return user
# user to user_id
def usertouserid(user):
    return user.id
# user to name
def usertoname(user):
    return user.name
# user to screen_name
def usertoscreenname(user):
    return user.screen_name
# user to location
def usertolocation(user):
    return user.location
# user to followers_count
def usertofollowerscount(user):
    return user.followers_count
# user to tweets_count
def usertotweetscount(user):
    return user.statuses_count


def appendtwitteruser(user):
    twitteruser.loc[twitteruser.shape[0]+1] = {"twitterUserId":usertouserid(user), 
                                               "name":usertoname(user), 
                                               "screenname":usertoscreenname(user), 
                                               "location":usertolocation(user), 
                                               "followersCount":usertofollowerscount(user), 
                                               "tweetsCount":usertotweetscount(user)}

Here we add basic attributes to companies, jobs, persons, employees.We initialize the CEOs of three companies as the employees. 

In [629]:
company.loc[company.shape[0]+1] = {"companyId":1, "companyName":"EtQ", "twitterUserId":usertouserid(anynametouser("etqsoftware"))}
job.loc[job.shape[0]+1] = {"jobId":1, "jobName":"Data Visualization Engineer"}
person.loc[person.shape[0]+1] = {"personId":1, "personName":"Rob Gremley", "twitterUserId":usertouserid(anynametouser("robgremley"))}
employee.loc[employee.shape[0]+1] = {"empId":1, "personId":1, "jobId":1, "companyId":1}
appendtwitteruser(anynametouser("robgremley"))
appendtwitteruser(anynametouser("etqsoftware"))
appendtwitteruser(anynametouser("DataVizSociety"))

In [630]:
company.loc[company.shape[0]+1] = {"companyId":2, "companyName":"Novartis", "twitterUserId":usertouserid(anynametouser("Novartis"))}
job.loc[job.shape[0]+1] = {"jobId":2, "jobName":"Data Scientist"}
person.loc[person.shape[0]+1] = {"personId":2, "personName":"Vasant Narasimhan", "twitterUserId":usertouserid(anynametouser("VasNarasimhan"))}
employee.loc[employee.shape[0]+1] = {"empId":2, "personId":2, "jobId":2, "companyId":2}
appendtwitteruser(anynametouser("VasNarasimhan"))
appendtwitteruser(anynametouser("Novartis"))
appendtwitteruser(anynametouser("NVSData"))

In [631]:
company.loc[company.shape[0]+1] = {"companyId":3, "companyName":"Bose", "twitterUserId":usertouserid(anynametouser("Bose"))}
job.loc[job.shape[0]+1] = {"jobId":3, "jobName":"Data Analytics and Insights Lead"}
person.loc[person.shape[0]+1] = {"personId":3, "personName":"Philip Hess", "twitterUserId":usertouserid(anynametouser("Bose"))}
employee.loc[employee.shape[0]+1] = {"empId":3, "personId":3, "jobId":3, "companyId":3}
appendtwitteruser(anynametouser("Bose"))
appendtwitteruser(anynametouser("Bose"))
appendtwitteruser(anynametouser("BDAnalyticsnews"))

In [632]:
# new tweet appending function
# tweet to tweet_id
def tweettotweetid(tweet):
    return tweet._json.get("id")
# tweet to user_id
def tweettouserid(tweet):
    return tweet._json.get("user").get("id")
# tweet to text
def tweettotext(tweet):
    return tweet._json.get("full_text")
# tweet to favorite_count
def tweettofavoritecount(tweet):
    return tweet._json.get("favorite_count")
# tweet to created_time
def tweettocreatedtime(tweet):
    return tweet.created_at



def appendtweet(twt):
    if tweet[tweet["tweetId"].isin([tweettotweetid(twt)])].shape[0] == 0:
        tweet.loc[tweet.shape[0]+1] = {"tweetId":tweettotweetid(twt), 
                                       "twitterUserId":tweettouserid(twt), 
                                       "content":tweettotext(twt), 
                                       "favoriteCount":tweettofavoritecount(twt), 
                                       "createdTime": tweettocreatedtime(twt)}
        for tag in tweettohashtaglist(twt):
            if hashtag[hashtag["text"].isin([tag])].shape[0] == 0:
                hashtag.loc[hashtag.shape[0]+1] = {"hashtagId":hashtag.shape[0]+1, "text":tag}
            tweetRhashtag.loc[tweetRhashtag.shape[0]+1] = {"trhId":tweetRhashtag.shape[0]+1, 
                                                           "tweetId":tweettotweetid(twt), 
                                                           "hashtagId":hashtag[hashtag["text"].isin([tag])].loc[:, ["hashtagId"]].values[0][0]}

In [633]:
# save employees' or companys' tweets in 30 days
def saveemployeecompanytweet(anyname):
    for twt0 in anynameto30daystweets(api, anyname):
            appendtweet(twt0)

In [634]:
try:
    saveemployeecompanytweet("etqsoftware")
    saveemployeecompanytweet("robgremley")
    saveemployeecompanytweet("DataVizSociety")
    saveemployeecompanytweet("Novartis")
    saveemployeecompanytweet("VasNarasimhan")
    saveemployeecompanytweet("NVSData")
    saveemployeecompanytweet("Bose")
    saveemployeecompanytweet("BDAnalyticsnews")
except tweepy.error.TweepError:
    pass

In [635]:
# save tweets based on the hashtag of jobName
def savehashtagtweet(jobName0):
    for twt0 in hashtagtotweetlist(jobName0):
        appendtweet(twt0)
        if twitteruser[twitteruser["twitterUserId"].isin([tweettouserid(twt0)])].shape[0] == 0:
            twitteruser.loc[twitteruser.shape[0]+1] = {"twitterUserId":tweettouserid(twt0), 
                                                       "name":usertoname(anynametouser(tweettouserid(twt0))), 
                                                       "screenname":usertoscreenname(anynametouser(tweettouserid(twt0))), 
                                                       "location":usertolocation(anynametouser(tweettouserid(twt0))), 
                                                       "followersCount":usertofollowerscount(anynametouser(tweettouserid(twt0))), 
                                                       "tweetsCount":usertotweetscount(anynametouser(tweettouserid(twt0)))}
            person.loc[person.shape[0]+1] = {"personId":person.shape[0]+1, 
                                             "personName":usertoscreenname(anynametouser(tweettouserid(twt0))), 
                                             "twitterUserId":tweettouserid(twt0)}
            people.loc[people.shape[0]+1] = {"personId":person.shape[0]}
        peopleRjob.loc[peopleRjob.shape[0]+1] = {
            "prjId":peopleRjob.shape[0]+1,
            "personId":person[person["twitterUserId"].isin([tweettouserid(twt0)])].loc[:, ["personId"]].values[0][0],
            "jobId":job[job["jobName"].isin([jobName0])].loc[:, ["jobId"]].values[0][0]
            }
        peopleRjob.drop_duplicates(subset=["personId","jobId"], keep='first', inplace=False)

In [705]:
try:
    savehashtagtweet("Data Visualization Engineer")
    
    
    
    
except tweepy.error.TweepError:
    pass

# Here we get csv file

In [637]:
# company = pd.DataFrame(columns=["companyId", "companyName", "twitterUserId"])
company

,companyId,companyName,twitterUserId
1,1,EtQ,90446356
2,2,Novartis,17226612
3,3,Bose,204074545


All information is showed, no 0, no null value, so it is already audited

In [638]:
#Make company a csv
company.to_csv(r'company.csv',index=False, encoding="utf-8-sig")

In [639]:
# people = pd.DataFrame(columns=["personId"])
people

,personId
1,4
2,5
3,6
4,7
5,8
6,9
7,10
8,11
9,12
10,13


In [640]:
people.shape

(19, 1)

In [641]:
people.isnull().values.any()

False

No null value, and no 0, we make costumer a csv

In [642]:
people.to_csv(r'people.csv',index=False, encoding="utf-8-sig")

In [643]:
# peopleRjob = pd.DataFrame(columns=["prjId", "personId", "jobId"])
peopleRjob

,prjId,personId,jobId
1,1,4,2
2,2,5,2
3,3,6,2
4,4,7,2
5,5,8,2
6,6,9,2
7,7,10,2
8,8,11,2
9,9,12,2
10,10,13,2


In [644]:
peopleRjob.shape

(19, 3)

In [645]:
peopleRjob.isnull().values.any()

False

In [646]:
peopleRjob.isnull().sum()

prjId       0
personId    0
jobId       0
dtype: int64

In [647]:
peopleRjob[peopleRjob.isnull().prjId == True]

,prjId,personId,jobId


In [648]:
peopleRjob=peopleRjob[peopleRjob.isnull().prjId == False]
peopleRjob.head()

,prjId,personId,jobId
1,1,4,2
2,2,5,2
3,3,6,2
4,4,7,2
5,5,8,2


In [649]:
peopleRjob.to_csv(r'peopleRjob.csv',index=False, encoding="utf-8-sig")

In [650]:
# job = pd.DataFrame(columns=["jobId", "jobName")
job

,jobId,jobName
1,1,Data Visualization Engineer
2,2,Data Scientist
3,3,Data Analytics and Insights Lead


All information is showed, no null values, no 0, so it is already audited

In [651]:
#make job a csv
job.to_csv(r'job.csv',index=False, encoding="utf-8-sig")

In [652]:
# hashtag = pd.DataFrame(columns=["hashtagId", "text"])
hashtag

,hashtagId,text
1,1,LED
2,2,crafts
3,3,qualitymanagement
4,4,quality
5,5,qualiyvision2020
6,6,qms
7,7,ehs
8,8,qualityvision2020
9,9,safety
10,10,etq


All information is showed, no null values, no 0, so it is already audited

In [653]:
hashtag.to_csv(r'hashtag.csv',index=False, encoding="utf-8-sig")

In [654]:
# person = pd.DataFrame(columns=["personId", "personName", "twitterUserId"])
person

,personId,personName,twitterUserId
1,1,Rob Gremley,41203376
2,2,Vasant Narasimhan,723488741348782080
3,3,Philip Hess,204074545
4,4,TekTaurus,3229087194
5,5,The_AIOps_Guy,1013777288335413248
6,6,femtech_,1143528514550927361
7,7,manisjohal,1215874082
8,8,GaryGenard,286178174
9,9,WebWindowsMkg,183263788
10,10,ReedAbend,1178423913757450253


In [655]:
person.shape

(22, 3)

In [656]:
person.isnull().values.any()

False

No null values, we make it a csv

In [657]:
person.to_csv(r'person.csv',index=False, encoding="utf-8-sig")

In [658]:
# employee = pd.DataFrame(columns=["empId", "personId", "jobId", "companyId"])
employee

,empId,personId,jobId,companyId
1,1,1,1,1
2,2,2,2,2
3,3,3,3,3


All information showed, no null values or 0, we make it a csv

In [659]:
employee.to_csv(r'employee.csv',index=False, encoding="utf-8-sig")

In [660]:
# tweet = pd.DataFrame(columns=["tweetId", "twitterUserId", "content", "favoriteCount", "createdTime"])
tweet

,tweetId,twitterUserId,content,favoriteCount,createdTime
1,1253396040213987329,90446356,"RT @FeatCustomers: Congratulations, @etqsoftwa...",0,2020-04-23 18:51:17
2,1250886805999804429,90446356,RT @SalesTechStar: @etqsoftware State Of Quali...,0,2020-04-16 20:40:29
3,1250126049800790016,90446356,RT @AbbottNews: UPDATE: We continue to expand ...,0,2020-04-14 18:17:31
4,1250125888785612804,90446356,RT @lumileds: Great work! This simple (and fun...,0,2020-04-14 18:16:52
5,1250042422828118016,90446356,"Webinar: Join ETQ Tues., 4/14, 11:00 a.m. PT f...",0,2020-04-14 12:45:12
6,1249691458032058368,90446356,"Webinar: Join ETQ Tues., 4/14, 11:00 a.m. PT f...",0,2020-04-13 13:30:36
7,1248619376720265216,90446356,"Webinar: Join ETQ Tues., 4/14, 11:00 a.m. PT f...",0,2020-04-10 14:30:32
8,1248312257722925062,90446356,"Webinar: Join ETQ Tues., 4/14, 11:00 a.m. PT f...",0,2020-04-09 18:10:09
9,1248266951807426560,90446356,RT @BillAulet: Great to see @WSJ getting the w...,0,2020-04-09 15:10:07
10,1247525163064647680,90446356,"Webinar: Join ETQ Tues., 4/14, 11:00 a.m. PT f...",0,2020-04-07 14:02:31


In [661]:
tweet.shape

(303, 5)

We see if there are any tweet with null value

In [662]:
tweet.isnull().values.any()

True

We see which part has null value

In [663]:
tweet.isnull().sum()

tweetId           0
twitterUserId     0
content          20
favoriteCount     0
createdTime       0
dtype: int64

We see the tweet with true value

In [664]:
tweet[tweet.isnull().content == True]

,tweetId,twitterUserId,content,favoriteCount,createdTime
284,1253707581324836864,3229087194,None,1,2020-04-24 15:29:14
285,1253700232774332419,1013777288335413248,None,0,2020-04-24 15:00:02
286,1253696980553007105,1143528514550927361,None,0,2020-04-24 14:47:07
287,1253690889941245957,1215874082,None,0,2020-04-24 14:22:55
288,1253685119308660737,286178174,None,1,2020-04-24 13:59:59
289,1253683856579969024,183263788,None,0,2020-04-24 13:54:58
290,1253681480070545409,1178423913757450253,None,0,2020-04-24 13:45:31
291,1253677664323461120,39560237,None,5,2020-04-24 13:30:22
292,1253671846920294400,85262756,None,0,2020-04-24 13:07:15
293,1253670167869100038,1200799908,None,4,2020-04-24 13:00:34


We delete the tweet with null value in content part

In [673]:
tweet=tweet[tweet.isnull().content == False]
tweet.head()

,tweetId,twitterUserId,content,favoriteCount,createdTime
1,1253396040213987329,90446356,"RT @FeatCustomers: Congratulations, @etqsoftwa...",0,2020-04-23 18:51:17
2,1250886805999804429,90446356,RT @SalesTechStar: @etqsoftware State Of Quali...,0,2020-04-16 20:40:29
3,1250126049800790016,90446356,RT @AbbottNews: UPDATE: We continue to expand ...,0,2020-04-14 18:17:31
4,1250125888785612804,90446356,RT @lumileds: Great work! This simple (and fun...,0,2020-04-14 18:16:52
5,1250042422828118016,90446356,"Webinar: Join ETQ Tues., 4/14, 11:00 a.m. PT f...",0,2020-04-14 12:45:12


In [666]:
#make tweet a csv
tweet.to_csv(r'tweet.csv',index=False, encoding="utf-8-sig")

In [667]:
# tweetRhashtag = pd.DataFrame(columns=["trhId", "tweetId", "hashtagId"])
tweetRhashtag

,trhId,tweetId,hashtagId
1,1,1250125888785612804,1
2,2,1250125888785612804,2
3,3,1237358851021852672,3
4,4,1235421984223309825,4
5,5,1235215179660566528,5
6,6,1235215179660566528,4
7,7,1235215179660566528,6
8,8,1235215179660566528,7
9,9,1234869932887879688,8
10,10,1234869932887879688,6


All information is showed, no null or 0, so it is already audited

In [668]:
#make tweetRhashtag a csv
tweetRhashtag.to_csv(r'tweetRhashtag.csv',index=False, encoding="utf-8-sig")

In [669]:
# twitteruser = pd.DataFrame(columns=["twitterUserId", "name", "screenname", "location", "followersCount", "tweetsCount"])
twitteruser

,twitterUserId,name,screenname,location,followersCount,tweetsCount
1,41203376,Rob Gremley,robgremley,,157,2
2,90446356,ETQ,etqsoftware,Offices in US & International,4273,5910
3,1086037534176473088,Data Visualization Society,DataVizSociety,,19877,1488
4,723488741348782080,Vas Narasimhan,VasNarasimhan,"Basel, Switzerland",13239,464
5,17226612,Novartis,Novartis,"Basel, Switzerland",266273,12211
6,767675409336897536,Mr Data Scientist,NVSData,"Melbourne, Victoria",3446,101745
7,204074545,Bose,Bose,"Framingham, MA",217421,10310
8,204074545,Bose,Bose,"Framingham, MA",217421,10310
9,1729180992,BigDataAnalyticsNews,BDAnalyticsnews,Hyderabad,3374,1501
10,3229087194,TekTaurus Education,TekTaurus,"New Jersey, USA",238,1026


In [670]:
twitteruser.shape

(28, 6)

In [671]:
twitteruser.isnull().values.any()

False

No null values, and some of tweetscount is 0. But 0 for tweetcount is a meaningful information, since it shows that this user has not tweeted. So we keep the 0

In [672]:
twitteruser.to_csv(r'tweetuser.csv',index=False, encoding="utf-8-sig")

# Here we generate our database

In [674]:
#import csv and sqlite3 library
import csv
import sqlite3

Here we read the data from the csv we have created

In [675]:
company = pd.read_csv(r'company.csv', encoding="utf-8-sig")
people = pd.read_csv('people.csv', encoding="utf-8-sig")
peopleRjob = pd.read_csv('peopleRjob.csv', encoding="utf-8-sig")
job = pd.read_csv('job.csv', encoding="utf-8-sig")
hashtag = pd.read_csv('hashtag.csv', encoding="utf-8-sig")
person = pd.read_csv('person.csv', encoding="utf-8-sig")
employee = pd.read_csv('employee.csv', encoding="utf-8-sig")
tweet = pd.read_csv('tweet.csv', encoding="utf-8-sig")
tweetRhashtag = pd.read_csv('tweetRhashtag.csv', encoding="utf-8-sig")
twitteruser = pd.read_csv('tweetuser.csv', encoding="utf-8-sig")



Make data lists

In [676]:
companyList = company.values.tolist()
peopleList = people.values.tolist()
peopleRjobList = peopleRjob.values.tolist()
jobList = job.values.tolist()
hashtagList = hashtag.values.tolist()
personList = person.values.tolist()
employeeList = employee.values.tolist()
tweetList = tweet.values.tolist()
tweetRhashtagList = tweetRhashtag.values.tolist()
twitteruserList = twitteruser.values.tolist()

In [677]:
con = sqlite3.connect('FinalProjectdatabse.db')

In [678]:
c = con.cursor()

We create tables here into our database

Create company table

In [679]:
c.execute("""CREATE TABLE company(
                   companyId INTEGER(20) PRIMARY KEY,
                   companyName VARCHAR(255),
                   twitterUserId INTEGER(255),
                   CONSTRAINT name_uni UNIQUE (companyId, twitterUserId)
                   
                   )""")

Create people table

In [680]:
c.execute("""CREATE TABLE people(
                   personId INTEGER(20) PRIMARY KEY,
                   CONSTRAINT name_uni UNIQUE (personId)
                   )""")

Create peopleRjob table

In [681]:
c.execute("""CREATE TABLE peopleRjob(
                   prjId INTEGER(20) PRIMARY KEY,
                   personId INTEGER(20),
                   jobId INTEGER(20),
                   CONSTRAINT name_uni UNIQUE (prjId)
                   )""")

Create job table

In [682]:
c.execute("""CREATE TABLE job(
                   jobId INTEGER(20) PRIMARY KEY,
                   jobName VARCHAR(255),
                   CONSTRAINT name_uni UNIQUE (jobid)
                   )""")

Create hashtage table

In [683]:
c.execute("""CREATE TABLE hashtag(
                   hashtagId INTEGER(20) PRIMARY KEY,
                   text VARCHAR(255),
                   CONSTRAINT name_uni UNIQUE (hashtagId, text)
                   )""")

Create person table

In [684]:
c.execute("""CREATE TABLE person(
                   personId INTEGER(20) PRIMARY KEY,
                   personName VARCHAR(255),
                   twitterUserId INTEGER(255),
                   CONSTRAINT name_uni UNIQUE (personId, twitterUserId)
                   )""")

Create employee tale

In [685]:
c.execute("""CREATE TABLE employee(
                   empId INTEGER(20) PRIMARY KEY,
                   personId INTEGER(20),
                   jobId INTEGER(20),
                   companyId INTEGER(20),
                   CONSTRAINT name_uni UNIQUE (empId)
                   )""")

Create tweet table

In [686]:
c.execute("""CREATE TABLE tweet(
                   tweetId INTEGER(255) PRIMARY KEY,
                   twitterUserId INTEGER(255),
                   content VARCHAR(255),
                   favoriteCount INTEGER(20),
                   createdTime DATE(20),
                   CONSTRAINT name_uni UNIQUE (tweetId)
                   )""")

Create tweetRhashtag table

In [687]:
c.execute("""CREATE TABLE tweetRhashtag(
                   trhId INTEGER(20) PRIMARY KEY,
                   tweetId INTEGER(255),
                   hashtagId INTEGER(20),
                   CONSTRAINT name_uni UNIQUE (trhId)
                   )""")

Create twitteruser table

In [688]:
c.execute("""CREATE TABLE twitteruser(
                   twitterUserId INTEGER(255) PRIMARY KEY,
                   name VARCHAR(255),
                   screenname VARCHAR(255),
                   location VARCHAR(255),
                   followersCount INTEGER(20),
                   tweetsCount INTEGER(20),
                   CONSTRAINT name_uni UNIQUE (twitterUserId)
                   )""")

# Insert values from our lists into tables

Here we insert values into our tables into databases

In [689]:
c.executemany("""INSERT or REPLACE INTO company VALUES(?,?,?)""", companyList)
c.executemany("""INSERT or REPLACE INTO people VALUES(?)""", peopleList)
c.executemany("""INSERT or REPLACE INTO peopleRjob VALUES(?,?,?)""", peopleRjobList)
c.executemany("""INSERT or REPLACE INTO job VALUES(?,?)""", jobList)
c.executemany("""INSERT or REPLACE INTO hashtag VALUES(?,?)""", hashtagList)
c.executemany("""INSERT or REPLACE INTO person VALUES(?,?,?)""", personList)
c.executemany("""INSERT or REPLACE INTO employee VALUES(?,?,?,?)""", employeeList)
c.executemany("""INSERT or REPLACE INTO tweet VALUES(?,?,?,?,?)""", tweetList)
c.executemany("""INSERT or REPLACE INTO tweetRhashtag VALUES(?,?,?)""", tweetRhashtagList)
c.executemany("""INSERT or REPLACE INTO twitteruser VALUES(?,?,?,?,?,?)""", twitteruserList)


# Test Case

Select all companies

In [690]:
c.execute("""SELECT *
             FROM company
          """)
for row in c.fetchall():
    print(row)

(1, 'EtQ', 90446356)
(2, 'Novartis', 17226612)
(3, 'Bose', 204074545)


Select all employees

In [691]:
c.execute("""SELECT *
             FROM employee
          """)
for row in c.fetchall():
    print(row)

(1, 1, 1, 1)
(2, 2, 2, 2)
(3, 3, 3, 3)


Select all the screennames from twitterusers

In [692]:
c.execute("""SELECT screenname
             FROM twitteruser
          """)
for row in c.fetchall():
    print(row)

('robgremley',)
('etqsoftware',)
('DataVizSociety',)
('VasNarasimhan',)
('Novartis',)
('NVSData',)
('Bose',)
('BDAnalyticsnews',)
('TekTaurus',)
('The_AIOps_Guy',)
('femtech_',)
('manisjohal',)
('GaryGenard',)
('WebWindowsMkg',)
('ReedAbend',)
('gurobi',)
('minuskeli',)
('TietoEVRYfi',)
('abhibisht89',)
('chidambara09',)
('StartGrowthHack',)
('akdm_bot',)
('JustBeMentalist',)
('fullNam35087976',)
('StaceyMGaines',)
('mrubioc',)
('bundesbank',)


# User case

Get twitteruser' userid order by their screenname in descending order

In [693]:

c.execute("""SELECT twitterUserId
             FROM twitteruser
             ORDER BY screenname DESC
          """)
for row in c.fetchall():
    print(row)

(41203376,)
(14472695,)
(85262756,)
(1215874082,)
(39560237,)
(985425300292022273,)
(1143528514550927361,)
(90446356,)
(737142202481016832,)
(784790730,)
(919744697551253505,)
(140743651,)
(183263788,)
(723488741348782080,)
(1200799908,)
(1013777288335413248,)
(3229087194,)
(715797742036062208,)
(23069519,)
(1178423913757450253,)
(17226612,)
(767675409336897536,)
(19539923,)
(286178174,)
(1086037534176473088,)
(204074545,)
(1729180992,)


Check the when a twitteruser tweet a post and the numbers of likes of the post and the content of the post by the tweetID. 

In [694]:

c.execute("""SELECT createdTime, favoriteCount, content
             FROM tweet
             WHERE tweetId = 1253396040213987329
          """)
for row in c.fetchall():
    print(row)

('2020-04-23 18:51:17', 0, "RT @FeatCustomers: Congratulations, @etqsoftware has been recognized as 'Best In Category' in Spring 2020 Quality Management Software Custo…")


Find the numbers of posts that tweeted by user 1013777288335413248	 within one day

In [697]:

c.execute("""SELECT COUNT(tweetId)
             FROM tweet
             WHERE twitterUserId = 1013777288335413248	 AND datetime(createdTime) > datetime('now','-1 day','localtime') 
          """)
for row in c.fetchall():
    print(row)

(0,)


Find 10 most popular hastags

In [698]:

c.execute("""SELECT hashtag.text, SUM(tweet.favoriteCount)
             FROM hashtag, tweet, tweetRhashtag
             WHERE hashtag.hashtagId=tweetRhashtag.hashtagId AND tweet.tweetId=tweetRhashtag.tweetId
             GROUP BY hashtag.hashtagId
             ORDER BY SUM(tweet.favoriteCount) DESC
             LIMIT 10
          """)
for row in c.fetchall():
    print(row)

('COVID19', 2616)
('dataviz', 598)
('DataVisualization', 589)
('Covid_19', 350)
('hydroxychloroquine', 301)
('datavisualization', 286)
('coronavirus', 270)
('VizResponsibly', 126)
('Datavisualization', 106)
('DataViz', 105)


Find the popular posts(the favourite counts > the average numbers)

In [699]:

c.execute("""SELECT *
             FROM tweet
             WHERE favoriteCount > (SELECT AVG(favoriteCount)
                                    FROM tweet)
          """)
for row in c.fetchall():
    print(row)

(1252981591791419392, 1086037534176473088, 'A reminder that today at 1PM PDT we are hosting a great conversation about making maps, hosted by @AlbertoCairo and featuring @mzdraper, @kennethfield &amp; @Elijah_Meeks. Join us for this live panel, the latest in our fireside chat series!\nhttps://t.co/GzlTZnP3mw', 25, '2020-04-22 15:24:25')
(1252231200028180481, 1086037534176473088, 'Welcome to Earth Week! To start us off, @duncangeere wrote about planetary boundaries, essentially a dashboard for the health of the entire planet. On the influential graphic’s history and how it might help us chart a sustainable way forward:\n\nhttps://t.co/NJ2sJJzZCE', 21, '2020-04-20 13:42:38')
(1251368893752373249, 1086037534176473088, 'What makes a great map? What makes a bad map? Join @mzdraper, @AlbertoCairo, @kennethfield and @Elijah_Meeks next Wednesday for a discussion about good maps, tips for designing them and cautionary tales about when maps go bad.\nhttps://t.co/GzlTZnP3mw', 162, '2020-04-18 04:

Find the employee with most fans

In [700]:

c.execute("""SELECT personName
             FROM person
             WHERE twitterUserId IN (SELECT twitterUserId
                                     FROM twitteruser
                                     WHERE followersCount = (SELECT MAX(followersCount)
                                                             FROM twitteruser
                                                             WHERE twitterUserId IN (SELECT twitterUserId
                                                                                     FROM person
                                                                                     WHERE personId IN (SELECT DISTINCT personId
                                                                                                        FROM employee
                                                                                                        )
                                                                                    )
                                                            )
                                     AND twitterUserId IN (SELECT twitterUserId
                                                           FROM person
                                                           WHERE personId IN (SELECT DISTINCT personId
                                                                              FROM employee
                                                                             )
                                                          )
                                     )
          """)
for row in c.fetchall():
    print(row)

('Philip Hess',)


Select the company with most fans

In [701]:

c.execute("""SELECT companyName
             FROM company
             WHERE twitterUserId IN (SELECT twitterUserId
                                     FROM twitteruser
                                     WHERE followersCount = (SELECT MAX(followersCount)
                                                             FROM twitteruser
                                                             WHERE twitterUserId IN (SELECT twitterUserId
                                                                                     FROM company
                                                                                    )
                                                             )
                                     AND twitterUserId IN (SELECT twitterUserId
                                                           FROM company
                                                          )
                                     )
          """)
for row in c.fetchall():
    print(row)

('Novartis',)


Select the company that provide job Data Scientist

In [702]:

c.execute("""SELECT companyName
             FROM company
             WHERE companyId IN (
                                SELECT DISTINCT companyId
                                FROM employee
                                WHERE jobId = (
                                                SELECT jobId
                                                FROM job
                                                WHERE jobName = "Data Scientist"
                                               )
                                )
          """)
for row in c.fetchall():
    print(row)

('Novartis',)


Select the top 10 locations of the Data Scientists

In [703]:

c.execute("""SELECT location, COUNT(*) AS count
             FROM twitteruser
             WHERE twitteruserId IN (
                                    SELECT twitteruserId
                                    FROM person
                                    WHERE personId IN (
                                                      SELECT personId
                                                      FROM peopleRjob
                                                      WHERE jobId = (
                                                                      SELECT jobId
                                                                      FROM job
                                                                      WHERE jobName = "Data Scientist"
                                                                     )
                                                     )
                                   )
             GROUP BY location
             ORDER BY count DESC
             LIMIT 10
          """)
for row in c.fetchall():
    print(row)

('Around the world ✈', 1)
('Berlin, Germany', 1)
('Boston', 1)
('Chiyoda-ku, Tokyo', 1)
('Devon', 1)
('Frankfurt am Main', 1)
('Helsinki', 1)
('Helsinki, Finland', 1)
('Madrid, Spain', 1)
('Mississippi', 1)


Show the job that Bose provides.

In [704]:

c.execute("""
             SELECT jobName
             FROM job
             WHERE jobId IN(
                             SELECT DISTINCT jobId
                             FROM employee
                             WHERE companyId = (
                                                SELECT companyId
                                                FROM company
                                                WHERE companyName = "Bose"
                                               )
                            )
          """)
for row in c.fetchall():
    print(row)

('Data Analytics and Insights Lead',)


# Contribution
## BY Minghao Ru/001088106: 50%
## BY Tong Yang/001302930 50%

# References





                                                
https://www.tableau.com/learn/articles/data-visualization
https://stackoverflow.com/questions/35415469/sqlite3-unique-constraint-failed-error
https://www.glassdoor.com/index.htm
https://developer.twitter.com/en/docs
https://twitter.com/Bose
https://twitter.com/Novartis
https://twitter.com/etqsoftware
https://github.com/nikbearbrown/INFO_6210/blob/master/Movie_DB_Example/Pandas_Data_Cleaning_Wrangling_and_Visualization.ipynb
https://github.com/nikbearbrown/INFO_6210/blob/master/Examples/A_Movie_Database.ipynb


# License

LICENSE:

MIT Liecnese

Copyright 2020(c) Minghao Ru, Tong Yang

You are free to copy and redistribute the material in any medium or format, remix, transform, and build upon the material for any purposes. 

